In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
#독립변수
ind_variables = ['age','wgBudam',   'wgHr',   'budam',   'rank',   'rcDist']
#범주형 변수
cat_variables = ['budam','rank']
#종속변수
dep_variables =['rcTime']

In [ ]:
#dummy 변수 생성
def categoricalToDummies(df, varName):
    result = pd.get_dummies(df[varName],prefix=varName)
    colnames = list(result.columns)
    for j in colnames:
        df[j]= result[j]
    del df[varName]
    return df

In [ ]:
#MLP 설정
#activation function - relu 사용 -> 다른 layer수정 가능 하다.
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) #행렬곱의 합
    layer_1 = tf.nn.relu(layer_1) #activation func
    # Hidden layer with RELU activation 
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) #행렬곱의 합
    layer_2 = tf.nn.relu(layer_2) #activation func
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out'] #행렬곱
    return out_layer

In [ ]:
# Parameters
learning_rate = 0.0001
training_epochs = 2000
batch_size = 128
display_step = 1

# Network Parameters
n_hidden_1 = 20 # 1st layer number of neurons
n_hidden_2 = 10 # 2nd layer number of neurons

n_input = 25 # number of features of my database
n_classes = 1 # one target value (float)

In [ ]:
# tf Graph input
x = tf.placeholder("float", [None, n_input]) #벡터? matrix들어옴
y = tf.placeholder("float", [None, n_classes])


data = pd.read_excel('./basic.xlsx')
# budam과 rank를 dummy변수화

for var in cat_variables:
    data = categoricalToDummies(data, var)
Y = data['rcTime']
X = data.drop('rcTime', axis=1)

#type 바꿔주기 및 train test spilt
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train = X_train.values.astype('float32')
y_train = y_train.values.reshape(-1,1).astype('float32')
X_test = X_test.values.astype('float32')
y_test = y_test.values.reshape(-1,1).astype('float32')
print(X_test.shape)

# Store layers weight & bias 
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.square(pred-y))
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer =  tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)




In [ ]:
# Initializing the variables
init = tf.initialize_all_variables() #초기화

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(X_train.shape[0] / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            randidx = np.random.randint(int(X_train.shape[0]), size=batch_size)
            batch_x = X_train[randidx, :]
            batch_y = y_train[randidx]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", \
                  "{:.9f}".format(avg_cost))
    # prediction = tf.argmax(y, 1)
    predict = pred.eval(feed_dict={x: X_test})
    print(predict)
    score = mean_squared_error(y_test, predict)
    print(score)
    print("Optimization Finished!")